In [13]:
import pandas as pd
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
cider_scorer = Cider()
bleu_scorer = Bleu(4)
meteor_scorer = Meteor()
rouge_scorer = Rouge()

In [7]:
!ls ../../model_cocopretrain_iotfinetune/*.csv

../../model_cocopretrain_iotfinetune/model_resnet152_5.csv


In [10]:
import pandas as pd

# Define file paths and their corresponding column names in a dictionary (flipped)
file_paths = {
    'vgg19': 'model_vgg19_10_prediction_IMERIT_2024_03_25_TEST_v5.csv',
    'rn152': 'model_resnet152_10_prediction_IMERIT_2024_03_25_TEST_v5.csv',
    'target': 'metadata_IMERIT_2024_03_25_TEST_v5.csv',
    'rn152_ft_highlr_failed': '../../model_cocopretrain_iotfinetune/model_resnet152_5.csv',
    'rn152_ft_1': '../../model_cocopretrain_iotfinetune_2/model_resnet152_1.csv',
    'rn152_ft_5': '../../model_cocopretrain_iotfinetune_2/model_resnet152_5.csv',
    'rn152_ft_10': '../../model_cocopretrain_iotfinetune_2/model_resnet152_10.csv',
}

# Initialize an empty list to store the dataframes
dataframes = []

# Loop through the file paths and load the CSV files
for column_name, path in file_paths.items():
    df = pd.read_csv(path).rename(columns={'text': column_name})
    dataframes.append(df)

# Start merging with the first dataframe
merged_df = dataframes[0]

# Loop through the rest of the dataframes and merge them on 'file_name'
for df in dataframes[1:]:
    merged_df = pd.merge(merged_df, df, on='file_name')

# Now `merged_df` contains the merged data from all the CSV files


In [11]:
merged_df 
merged_df.to_csv("caption_comparison.csv")

In [14]:
for model_name in file_paths.keys():
    print('-'*20, model_name.upper(), '-'*20)
    refer_captions = {}
    predc_captions = {}
    for i in range(len(merged_df)):
        refer_captions[merged_df.iloc[i]['file_name']] = [str(merged_df.iloc[i][model_name])]
        predc_captions[merged_df.iloc[i]['file_name']] = [str(merged_df.iloc[i]['target'])]
    c, _ = cider_scorer.compute_score(refer_captions, predc_captions); print("CIDEr", c)
    b, _ = bleu_scorer.compute_score(refer_captions, predc_captions); print('BLEU', b)
    r, _ = rouge_scorer.compute_score(refer_captions, predc_captions); print('ROUGE', r, '\n')

-------------------- VGG19 --------------------
CIDEr 0.036490947241140666
{'testlen': 7167, 'reflen': 11664, 'guess': [7167, 6185, 5203, 4221], 'correct': [959, 51, 0, 0]}
ratio: 0.6144547325102354
BLEU [0.07144620078661279, 0.01773592217667285, 3.184079383057296e-08, 4.4952987680319783e-11]
ROUGE 0.09457508711908663 

-------------------- RN152 --------------------
CIDEr 0.038019906963850575
{'testlen': 7167, 'reflen': 11985, 'guess': [7167, 6185, 5203, 4221], 'correct': [984, 44, 3, 1]}
ratio: 0.597997496871039
BLEU [0.07009776689995645, 0.01595628637425042, 0.0042162451443005285, 0.0017352105308910395]
ROUGE 0.09702593981792966 

-------------------- TARGET --------------------
CIDEr 10.0
{'testlen': 7167, 'reflen': 7167, 'guess': [7167, 6185, 5203, 4221], 'correct': [7167, 6185, 5203, 4221]}
ratio: 0.9999999999998604
BLEU [0.9999999999997208, 0.9999999999997097, 0.9999999999996958, 0.9999999999996777]
ROUGE 1.0 

-------------------- RN152_FT_HIGHLR_FAILED --------------------
CID